In [1]:
import sys
sys.path.insert(1, '../../../scripts/')
from s3_support import *

import pandas as pd

In [2]:
date_grpd = redshift_query_read("select count(date) as count, date from org_websites group by date order by date desc")
date_grpd.head(2)

,count,date
0,41373,2020-08-15
1,40873,2020-08-01


In [3]:
last_2_dates = date_grpd.head(2)['date'].tolist()
df = redshift_query_read("select * from org_websites where date in ('{}', '{}')".format(last_2_dates[0], last_2_dates[1]))

In [4]:
df.head(3)

,org,date,calls_to_action,iframe_source_blackbaud,iframe_source_classy,iframe_source_donordrive,iframe_source_engagingnetworks,iframe_source_facebook,iframe_source_giveeffect,iframe_source_givelively,...,script_source_onecause,script_source_paypal,script_source_qgiv,script_source_stripe,script_source_twitter,url,word_count,iframe_source_securegive,link_target_securegive,script_source_securegive
0,13,2020-08-01,8,0,0,0,0,0,0,0,...,0,0,0,0,0,http://www.viste.org,2594,0,0,0
1,33,2020-08-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,http://www.yeshekhorlo.org,2700,0,0,0
2,39,2020-08-01,2,0,0,0,0,0,0,0,...,0,0,0,0,0,https://www.givecf.org/united-community-relief...,1328,0,0,0


In [5]:
df.groupby('date')['org'].count()

date
2020-08-01    40873
2020-08-15    41373
Name: org, dtype: int64

In [6]:
sources = [c.replace('iframe_source_', '') for c in df.columns if 'iframe' in c]
sources

['blackbaud',
 'classy',
 'donordrive',
 'engagingnetworks',
 'facebook',
 'giveeffect',
 'givelively',
 'instagram',
 'mobilecause',
 'networkforgood',
 'onecause',
 'paypal',
 'qgiv',
 'stripe',
 'twitter',
 'securegive']

In [7]:
source_presence_data = []
for source in sources:
    rel_cols = [c for c in df.columns if source in c]
    presence = df[rel_cols].sum().sum() / float(len(df))
    source_presence_data.append({
        'source': source,
        'presence': presence * 100.
    })
pd.DataFrame(source_presence_data)[['source', 'presence']]

,source,presence
0,blackbaud,0.503368
1,classy,0.457165
2,donordrive,0.362328
3,engagingnetworks,0.000000
4,facebook,171.746954
5,giveeffect,0.000000
6,givelively,0.331931
7,instagram,93.656834
8,mobilecause,0.195754
9,networkforgood,2.448751


In [8]:
df_refs = df.copy()

source_references = []
for source in sources:
    source_references.append("ref_{}".format(source))
    df_refs["ref_{}".format(source)] = df_refs[[c for c in df_refs.columns if source in c]].sum(axis=1)

In [9]:
df_daily = df_refs[['org', 'date'] + source_references].groupby(['org', 'date'])[source_references].sum().reset_index()

In [10]:
df_daily = df_daily[df_daily['date'].isin(df_daily['date'].sort_values().unique()[-2:])]
len(df_daily['org'].unique().tolist()), df_daily['date'].unique().tolist()

(3826, [1596240000000000000, 1597449600000000000])

In [11]:
df_daily.head(3)

,org,date,ref_blackbaud,ref_classy,ref_donordrive,ref_engagingnetworks,ref_facebook,ref_giveeffect,ref_givelively,ref_instagram,ref_mobilecause,ref_networkforgood,ref_onecause,ref_paypal,ref_qgiv,ref_stripe,ref_twitter,ref_securegive
0,6,2020-08-01,0,0,0,0,2,0,0,2,0,0,0,0,3,0,2,0
1,6,2020-08-15,0,0,0,0,2,0,0,2,0,0,0,0,3,0,2,0
2,13,2020-08-01,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0


### trying groupby

In [12]:
df_daily_diffs = df_daily.sort_values('date', ascending=True).groupby('org')[source_references].diff().reset_index()

In [13]:
df_daily_diffs = df_daily_diffs.dropna(how='all')
df_daily_diffs[df_daily_diffs['ref_qgiv']<0].sort_values('ref_qgiv').head()

,index,ref_blackbaud,ref_classy,ref_donordrive,ref_engagingnetworks,ref_facebook,ref_giveeffect,ref_givelively,ref_instagram,ref_mobilecause,ref_networkforgood,ref_onecause,ref_paypal,ref_qgiv,ref_stripe,ref_twitter,ref_securegive
4037,26390,0.0,0.0,0.0,0.0,-229.0,0.0,0.0,-171.0,0.0,0.0,0.0,0.0,-926.0,0.0,-226.0,0.0
6667,9616,0.0,0.0,0.0,0.0,-69.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,-579.0,0.0,-32.0,0.0
3887,24691,0.0,0.0,0.0,0.0,-216.0,0.0,0.0,-52.0,0.0,0.0,0.0,0.0,-416.0,0.0,-208.0,0.0
6278,18952,0.0,0.0,0.0,0.0,-32.0,0.0,0.0,-31.0,0.0,0.0,0.0,0.0,-289.0,0.0,-30.0,0.0
6775,13444,0.0,0.0,0.0,0.0,-77.0,0.0,0.0,-66.0,0.0,0.0,0.0,0.0,-276.0,0.0,0.0,0.0


In [14]:
df_daily.sort_values('date', ascending=True).groupby('org')['ref_qgiv'].diff().dropna().sort_values()[:5]

26390   -926.0
9616    -579.0
24691   -416.0
18952   -289.0
13444   -276.0
Name: ref_qgiv, dtype: float64

### trying iteration

In [12]:
data = None
for o in df_daily['org'].unique().tolist():
    _df = df_daily[df_daily['org']==o].sort_values('date', ascending=True)
    entry = _df[source_references].diff()
    entry['org'] = o
    
    if data is None:
        data = entry
    else:
        data = data.append(entry)

In [13]:
data.dropna().sort_values('ref_qgiv').head(10)

,ref_blackbaud,ref_classy,ref_donordrive,ref_engagingnetworks,ref_facebook,ref_giveeffect,ref_givelively,ref_instagram,ref_mobilecause,ref_networkforgood,ref_onecause,ref_paypal,ref_qgiv,ref_stripe,ref_twitter,ref_securegive,org
735,0.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,14.0,-75.0,0.0,0.0,0.0,12225
5316,0.0,0.0,0.0,0.0,-54.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-69.0,0.0,-49.0,0.0,443606
5508,0.0,0.0,0.0,0.0,-128.0,0.0,0.0,-8.0,0.0,0.0,0.0,0.0,-47.0,0.0,-128.0,0.0,443807
2401,0.0,0.0,0.0,0.0,-23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-29.0,0.0,-23.0,0.0,430205
7206,0.0,0.0,0.0,0.0,-165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-23.0,0.0,-42.0,0.0,445464
552,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-19.0,0.0,-7.0,0.0,1643
359,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-19.0,0.0,0.0,0.0,847
2459,0.0,0.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,-18.0,0.0,-2.0,0.0,431194
113,0.0,0.0,0.0,0.0,-388.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-16.0,0.0,-497.0,0.0,243
4782,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,-12.0,0.0,-10.0,0.0,443137


In [15]:
df_daily.groupby('org')['date'].count().mean()

1.9692389286661363